In [12]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
from scipy.signal import find_peaks, peak_widths, peak_prominences
from BaselineRemoval import BaselineRemoval

In [13]:
#Import dataset1 for either training or testing
global_dir_path = "C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data"
#4wks
# dir_path = global_dir_path + '/2023_0809_0810_Breath_Eml4Alk_4wks_multiplex/Spectra'
#5wks
# dir_path = global_dir_path + '/2023_0816_0817_Breath_Eml4Alk_5wks_multiplex/All samples_spectra'
#6wks
# dir_path = global_dir_path + '/2023_May_June_Eml4Alk_6to8wks_spectra/2023_05_19_Breath_Eml4Alk_6wks_calibrated_spectra'

#6wks August probably not use because it is the failed longitudinal
#dir_path = global_dir_path + '/2023_08_24_Breath_EmlAlk_6wks_multiplex/Spectra'

#4wks October
# dir_path = global_dir_path + '/2023_10_19_Breath_Eml4Alk_4wk_5plex/Spectra'

#5wks October
# dir_path = global_dir_path + '/2023_10_26_Breath_Eml4Alk_5wks_cohort2/spectra'

#6wks October also failed longitudinal but at least some of the signals were correct (also more advanced tumors)
# dir_path = global_dir_path + '/2023_10_23_Breath_Eml4-Alk_6wks_cohort1/spectra'

#6wks Jan2024
# dir_path = global_dir_path + '/2024_0116_0117_Breath_Eml4aLK_6w_endo'

#5wks Feb2024
# dir_path = global_dir_path + '/2024_02_09_Eml4-Alk_5wks_multiplex/Spectra'

#6wks Feb2024
dir_path = global_dir_path + '/2024_02_16_Eml4Alk_6wks_multiplex/spectra'

dir_list = sorted(os.listdir(dir_path))
print(dir_list)

# dir_list2 = sorted(os.listdir(dir_path2))
# print(dir_list2)

['2024_02_16_W6_EA4_1.txt', '2024_02_16_W6_EA4_1_blank.txt', '2024_02_16_W6_EA4_2.txt', '2024_02_16_W6_EA4_2_blank.txt', '2024_02_16_W6_EA4_3.txt', '2024_02_16_W6_EA4_3_blank.txt', '2024_02_16_W6_EA4_4.txt', '2024_02_16_W6_EA4_4_blank.txt', '2024_02_16_W6_EA5_1.txt', '2024_02_16_W6_EA5_1_blank.txt', '2024_02_16_W6_EA5_2.txt', '2024_02_16_W6_EA5_2_blank.txt', '2024_02_16_W6_EA5_3.txt', '2024_02_16_W6_EA5_3_blank.txt', '2024_02_16_W6_EA5_4.txt', '2024_02_16_W6_EA5_4_blank.txt', '2024_02_16_W6_EA5_5.txt', '2024_02_16_W6_EA5_5_blank.txt', '2024_02_16_W6_Healthy2_1.txt', '2024_02_16_W6_Healthy2_1_blank.txt', '2024_02_16_W6_Healthy2_2.txt', '2024_02_16_W6_Healthy2_2_blank.txt', '2024_02_16_W6_Healthy2_3.txt', '2024_02_16_W6_Healthy2_3_blank.txt', '2024_02_16_W6_Healthy2_4.txt', '2024_02_16_W6_Healthy2_4_blank.txt', '2024_02_16_W6_Healthy2_5.txt', '2024_02_16_W6_Healthy2_5_blank.txt', 'Peaks', 'Spectra not used']


In [14]:
def get_prominence(d):
    peaks, _ = find_peaks(signal_corr, distance=d, height=HEIGHT, width=1.2) #change, add width
    prominences = peak_prominences(signal_corr, peaks)[0]

    dictionary = {}
    for i in prominences:
        if int(i) not in dictionary:
            dictionary[int(i)] = 1
        else: 
            dictionary[int(i)] += 1 

    result = max(dictionary.values())
    prom = []
    for i in dictionary:
        if dictionary[i] == result:
            prom.append(i)

    best_p = max(prom)

    return best_p

In [15]:
def view_peak(d):
    peaks, _ = find_peaks(signal_corr, distance=d, height=HEIGHT, width=1.2) #change, add width
    plt.plot(m_to_z, signal_corr, label='data (baseline corrected)')
    plt.plot(m_to_z[peaks], signal_corr[peaks], "x", label='peaks')
    plt.xlabel('m/z')
    plt.ylabel('Intensity (a.u.)')
    #plt.ylim(0, 1000)
    #plt.xlim(13,50)
    plt.legend()
    plt.show()
    
def get_mz(d):
    peaks, _ = find_peaks(signal_corr, distance=d, height=HEIGHT, width=1.2) #change, add width
    mz = np.zeros(len(peaks))
    for i in range(len(peaks)):
        index = peaks[i]
        mz[i] = m_to_z[index]
    return mz

def min_dist(arr):
    min_dist = 100000
    for i in range(len(arr) - 1):
        if arr[i + 1] - arr[i] < min_dist:
            min_dist = arr[i + 1] - arr[i]
    return min_dist

In [16]:
#Background substraction
for i in range(0, len(dir_list), 2):
    if ".txt" in dir_list[i]:
        name = dir_path + '/' + dir_list[i]
        print(name) # to make sure the bk are the blank files using this method

        #for 4wk, 5wk August 6wks, Jan2024 6wks
#         bk = pd.read_csv(name, sep='\t')
#         df = pd.read_csv(dir_path + '/' + dir_list[i+1], sep='\t')

        #for 6 wks May, Feb2024 5wks, and Feb2024 6wks
        df = pd.read_csv(name, sep='\t')
        bk = pd.read_csv(dir_path + '/' + dir_list[i+1], sep='\t')
        
        m_to_z = df.iloc[:,2]
        signal = df.iloc[:,3]
        bk_signal = bk.iloc[:,3]

        sub_signal = signal-bk_signal #removes background signal
        sub_signal = sub_signal.clip(lower=0) #removes negatives
        base=BaselineRemoval(sub_signal)
        signal_corr=base.ZhangFit() #ensures peaks all have the same baseline
        HEIGHT = 15

        #Find min_d
        arr = []
        for d in range(1, 300): #change, increase range
            arr.append((d, min_dist(get_mz(d))))
        arr.sort(key = lambda x: x[1])

        for j in range(len(arr)-1):
            if arr[j][1] < 0.03 and arr[j][1] > 0.01: #change, distance 0.05 m/z instead of 0.01
                #adjust the range to 0.06 because some files cannot find min_d if the range is too small
                min_d = arr[j][0]
                break
        
        peaks, _ = find_peaks(signal_corr, distance=min_d, height=HEIGHT, width=1.2, prominence=get_prominence(min_d))
        
        peak_m_z = [m_to_z[peak] for peak in peaks] #THIS IS CORRECT 
        intarray = [signal_corr[peak] for peak in peaks] #THIS IS CORRECT
             
        zip_result = zip(peak_m_z, intarray)
        columns=['m/z', 'Intensity (corr)']
        df1 = pd.DataFrame(zip_result, columns=columns)

        #for 4wk, 5wk, August 6wk, and Jan2024 6wk
#         aucfilename = dir_path + '/Peaks/' + dir_list[i+1]  #NEED TO CREATE NEW FOLDER

        #for 6 wks May, Feb2024 5wks, and Feb2024 6wks
        aucfilename = dir_path + '/Peaks/' + dir_list[i]  #NEED TO CREATE NEW FOLDER

        aucfilename = aucfilename.replace(".txt","_peaks.xlsx") 
        df1.to_excel(aucfilename)

C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_02_16_Eml4Alk_6wks_multiplex/spectra/2024_02_16_W6_EA4_1.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_02_16_Eml4Alk_6wks_multiplex/spectra/2024_02_16_W6_EA4_2.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_02_16_Eml4Alk_6wks_multiplex/spectra/2024_02_16_W6_EA4_3.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_02_16_Eml4Alk_6wks_multiplex/spectra/2024_02_16_W6_EA4_4.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_02_16_Eml4Alk_6wks_multiplex/spectra/2024_02_16_W6_EA5_1.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Lung cancer/Headspace_data/2024_02_16_Eml4Alk_6wks_multiplex/spectra/2024_02_16_W6_EA5_2.txt
C://Users/chris/OneDrive/Desktop/MIT postdoc project